### Clone the yolov5 source

In [ ]:
!git clone https://github.com/ultralytics/yolov5

### Modify the requirements of yolo in case of incompatibility

In [ ]:
with open("/kaggle/working/yolov5/requirements.txt", "r") as file:
    content = file.read()
    content = content.replace("numpy>=1.18.5", "numpy==1.22")
    with open("/kaggle/working/yolov5/requirements.txt", "w") as f:
        f.write(content)

### Create a python 3.9 env on kaggle for GPU

In [ ]:
!conda create -n py39 python=3.9 --yes

!source activate py39

!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/py39/bin/python3 /opt/conda/bin/python3

!sudo rm /opt/conda/bin/python3.7
!sudo ln -sf /opt/conda/envs/py39/bin/python3 /opt/conda/bin/python3.7

!sudo rm /opt/conda/bin/python
!sudo ln -s /opt/conda/envs/py39/bin/python3 /opt/conda/bin/python

!export PATH="/opt/conda/envs/py39/bin:$PATH"

!echo 'export PATH="/opt/conda/envs/py39/bin:$PATH"' >> ~/.bashrc

!source ~/.bashrc

import os
os.environ['PATH'] += ":/opt/conda/envs/py39/bin"

### Install requirements for yolov5

In [ ]:
!python -m pip install -U -r /kaggle/working/yolov5/requirements.txt

### Install additional required libraries

In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install tensorboard

### Create a config file for data description

In [ ]:
import os
from sklearn.model_selection import train_test_split

initial_train_path = "/kaggle/input/ada-image-recognition-fiber/dataset/images/train/"
aug_train_path = "/kaggle/input/data-fiber-augs/dataset_aug/images/aug_train/"

initial_train = os.listdir(initial_train_path)
initial_train = list(map(lambda x: initial_train_path + x, initial_train))

aug_train = os.listdir(aug_train_path)
aug_train = list(map(lambda x: aug_train_path + x, aug_train))

all_train_imgs = initial_train + aug_train

train_imgs, val_imgs = train_test_split(all_train_imgs, train_size=0.9, random_state=1)

train_img_list = "/kaggle/working/train.txt"
val_img_list = "/kaggle/working/val.txt"

for imgs, imgs_list_file in zip([train_imgs, val_imgs], [train_img_list, val_img_list]):
    with open(imgs_list_file, 'w') as file:
        for img in imgs:
            file.write(img + "\n")
yaml_path = "/kaggle/working/custom.yaml"
yaml_content = f""" 
train: {train_img_list}
val: {val_img_list}
test: /kaggle/input/ada-image-recognition-fiber/dataset/images/test

nc: 5

# class names
names: 
  0: Screw,
  1: Foam
  2: Plastic cover
  3: Tie-wrap, 
  4: Rubbers
"""

with open(yaml_path, 'w') as file:
    file.write(yaml_content)

### Modify training hyperparameters

In [ ]:
with open("/kaggle/working/yolov5/data/hyps/hyp.scratch-high.yaml", "r") as file:
    content = file.read()
    content = content.replace("lr0: 0.01", "lr0: 0.05")
    with open("/kaggle/working/yolov5/data/hyps/hyp.scratch-high.yaml", "w") as f:
        f.write(content)

### Transfer learning

In [ ]:
!python /kaggle/working/yolov5/train.py --data /kaggle/working/custom.yaml --weights yolov5l.pt --epochs 40 --batch 64 --freeze 10 --optimizer Adam
#!python /kaggle/working/yolov5/train.py --weights /kaggle/input/model-40-epochs/runs/train/exp/weights/last.pt --data /kaggle/working/custom.yaml --hyp /kaggle/working/yolov5/data/hyps/hyp.scratch-high.yaml --epochs 40 --batch 16 --freeze 10 --optimizer SGD
#!python /kaggle/working/yolov5/train.py --data /kaggle/working/custom.yaml --weights /kaggle/input/model-40-epochs/runs/train/exp/weights/last.pt --epochs 20 --batch 64 --freeze 10 --optimizer Adam
#!python /kaggle/working/yolov5/train.py --data /kaggle/working/custom.yaml --weights yolov5n.pt --epochs 2 --batch 64 --freeze 10 --optimizer Adam